In [ ]:
%load_ext autoreload 
%autoreload 2

# Initial exploration of full training LSTM simple 01


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf

from src.modeling.prototyping.lstm_protype_simple import validated_tf
from src.modeling.prototyping.lstm_simple_initial_training import build_path

### Tensorflow spec

In [ ]:
validated_tf()

### model specs

In [ ]:
LTSM_1_PATH = '/projects/p31961/dopamine_modeling/results/models/lstm_simple_initial_training_with_downsampling'

In [ ]:
ltsm = tf.keras.models.load_model(LTSM_1_PATH)

In [ ]:
ltsm.summary()

* data was downsampled during preprocessing selecting every 100th data point

In [ ]:
PATH_TO_DATA = '/projects/p31961/gaby_data/aggregated_data/data_pipeline/datasets'


In [ ]:
X_test_path = build_path(PATH_TO_DATA, 'X_test_downsampled')
y_test_path = build_path(PATH_TO_DATA, 'y_test_downsampled')

In [ ]:
X_test = pd.read_parquet(X_test_path)
y_test = pd.read_parquet(y_test_path)

In [ ]:
predicted_signal = ltsm.predict(X_test)

In [ ]:
predicted_signal

### reformating and data aggregation


In [ ]:
def combine_data(df, true_signal, infered_signal):
    cols_to_group = ['day', 'time', 'event_cue', 'event_shock', 'sensor_D1', 'sensor_D2', 'sensor_DA']
    agg_diict = {'true_signal': ['mean', 'sem'],
             'predicted_signal': ['mean', 'sem']
             }
    def flatten_df(df):
        df.columns = ['_'.join(col) for col in df.columns.values]
        df = df.reset_index()
        return df
    def strip_cols(df):
        df.columns = [col.rstrip('_') for col in df.columns]
        return df
    return(
        df.assign(
            true_signal=true_signal, 
            predicted_signal=infered_signal)
        .reset_index(drop=True)
        .groupby(by = cols_to_group, as_index = False).agg(agg_diict)
        .pipe(flatten_df)
        .pipe(strip_cols)
        .drop(columns = 'index')

    )

In [ ]:
combined_data = combine_data(df = X_test, true_signal=y_test, infered_signal=predicted_signal)
combined_data


In [ ]:

test_data = combined_data.query('day==6 & event_cue==0 & event_shock==1 & sensor_DA==1')
test_data.isna().sum()

# Predicted vs true signal
* day 6
* shock
* dopamine

In [ ]:
# sns.lineplot(data=test_data, x='time', y='true_signal_mean', color='blue')
sns.lineplot(data=test_data, x='time', y='true_signal_mean', color='blue', linewidth=0.5)
sns.lineplot(data=test_data, x='time', y='predicted_signal_mean', color='black', linewidth=0.5)
# plt.fill_between(test_data['time'], test_data['predicted_signal_mean'] - test_data['predicted_signal_sem'], test_data['predicted_signal_mean'] + test_data['predicted_signal_sem'], alpha=1, color='black')

# Summary 
the data is clearly incorrect. I think data is downsampled too much, I have preprocessed the data gain sampling every 10th data point and retrained to LSTM simple 02

_____

# Validation of LSTM_02_simple retran with smaller downsample (25)


In [ ]:
LTSM_2 = '/projects/p31961/dopamine_modeling/results/models/LSTM_02_simple'

In [ ]:
lstm_2 = tf.keras.models.load_model(LTSM_2)

## LTSM summary

In [ ]:
lstm_2.summary()

### read data into pandas

In [ ]:
X_test_25_path = build_path(path_to_data = PATH_TO_DATA, data_set_prefix='5_day_training_gaby_ds25_',dataset_name='X_test_downsampled')
y_test_25_path = build_path(path_to_data = PATH_TO_DATA, data_set_prefix='5_day_training_gaby_ds25_',dataset_name='y_test_downsampled')

In [ ]:
X_test_25 = pd.read_parquet(X_test_25_path)
y_test_25 = pd.read_parquet(y_test_25_path)


### Signal inference

In [ ]:
predicted_signal_25 = lstm_2.predict(X_test_25)
predicted_signal_25

In [ ]:
combined_data_25 = combine_data(df = X_test_25, true_signal=y_test_25, infered_signal=predicted_signal_25)
combined_data_25

In [ ]:
day6_cue_da = combined_data_25.query('day==6 & event_cue==1 & event_shock==0 & sensor_DA==1')
day6_cue_da.isna().sum()
# day6_cue_da

In [ ]:
def plot_signal(y):
    sns.lineplot(data=day6_cue_da, x='time', y=f'{y}_mean', color='black', linewidth=0.5)
    plt.fill_between(day6_cue_da['time'], day6_cue_da[f'{y}_mean'] - day6_cue_da[f'{y}_sem'], day6_cue_da[f'{y}_mean'] + day6_cue_da[f'{y}_sem'], alpha=.2, color='black')
plot_signal('true_signal')
plot_signal('predicted_signal')

# what is wrong with the data?`

In [3]:
RAW_PATH = '/projects/p31961/gaby_data/aggregated_data/aggregated_data.parquet.gzp'
raw_data = pd.read_parquet(RAW_PATH)
raw_data

,mouse_id,day,event,sensor,time,trial,signal
0,142_237,1,avoid,D1,-25.000000,0,0.955051
1,142_237,1,avoid,D1,-24.998034,0,0.956232
2,142_237,1,avoid,D1,-24.996067,0,0.957714
3,142_237,1,avoid,D1,-24.994102,0,0.959506
4,142_237,1,avoid,D1,-24.992136,0,0.961616
...,...,...,...,...,...,...,...
22884,313_255,7,shock,DA,19.992136,0,0.741634
22885,313_255,7,shock,DA,19.994102,0,0.713201
22886,313_255,7,shock,DA,19.996067,0,0.684301
22887,313_255,7,shock,DA,19.998034,0,0.654951


In [ ]:
raw_ds25 = raw_data[::25]
raw_ds25

In [ ]:
def filter_data(df):
    def flatten_df(df):
        df.columns = ['_'.join(col) for col in df.columns.values]
        df = df.reset_index()
        return df
    def strip_cols(df):
            df.columns = [col.rstrip('_') for col in df.columns]
            return df
        
    group_by = ['time', 'sensor', 'mouse_id', 'day', 'event']
    agg_dict = {'signal': ['mean']}

    grouped= (
        df.groupby(by=group_by, as_index=False).agg(agg_dict)
        .pipe(flatten_df)
        .pipe(strip_cols)
        .drop(columns = 'index') 
    )
    return (grouped
            .query('day==1 & event=="cue" & sensor=="DA"')
            .groupby(by = ['time'], as_index=True).agg({'signal_mean': ['mean', 'sem']})
            .pipe(flatten_df)
            .pipe(strip_cols)
    )

d1_cue_da_ds = filter_data(raw_ds25)


In [ ]:
d1_cue_da_ds

In [ ]:
d1_cue_da_raw = filter_data(raw_data)

In [ ]:
d1_cue_da_raw
correct_down_sample = d1_cue_da_raw[::100]

In [ ]:
def plot_data(df, title):
    sns.lineplot(data=df, x='time', y='signal_mean_mean',linewidth=0.2)
    # change y axsis to be -2 to 2
    plt.fill_between(df['time'], df['signal_mean_mean'] - df['signal_mean_sem'], df['signal_mean_mean'] + df['signal_mean_sem'], alpha=.2, color='black')
    plt.ylim(-2, 2)
    plt.title(title)
    
# plot_data(df = d1_cue_da_raw, title='raw data')

In [ ]:

plot_data(df = d1_cue_da_ds, title='downsampled data')


In [ ]:
plot_data(df = correct_down_sample, title='correct downsampled data')

# error in down sampling
* I made a mistake in when I introduced the downsampling for this data. I think I need to introduce it after grouping and processing right before training? The below is an interative process for finding the correct order of operations

In [ ]:
def filter_data_update(df):
    def flatten_df(df):
        df.columns = ['_'.join(col) for col in df.columns.values]
        df = df.reset_index()
        return df
    def strip_cols(df):
            df.columns = [col.rstrip('_') for col in df.columns]
            return df
        
    group_by = ['time', 'sensor', 'day', 'event']
    agg_dict = {'signal': ['mean', 'sem']}

    grouped= (
        df.groupby(by=group_by, as_index=False).agg(agg_dict)
        .pipe(flatten_df)
        .pipe(strip_cols)
        .drop(columns = 'index') 
    )
    return (grouped
            # .query('day==1 & event=="cue" & sensor=="DA"')
            # .groupby(by = ['time', ], as_index=True).agg({'signal_mean': ['mean', 'sem']})
            # .pipe(flatten_df)
            # .pipe(strip_cols)
    )


grouped_data_update = filter_data_update(raw_data)

In [ ]:
day1_cue_da = grouped_data_update.query('day==1 & event=="cue" & sensor=="DA"')
day1_cue_da

In [ ]:
def plot_data_update(df, title):
    sns.lineplot(data=df, x='time', y='signal_mean',linewidth=0.2)
    # change y axsis to be -2 to 2
    plt.fill_between(df['time'], df['signal_mean'] - df['signal_sem'], df['signal_mean'] + df['signal_sem'], alpha=.2, color='black')
    plt.ylim(-2, 2)
    plt.title(title)
plot_data_update(df = day1_cue_da, title='raw data updated filter')

## I beleive the downsampling after the aggregated and melted dataframe is the problem. 
* possible soluton - convert time to timeseries in padas -> resample 


In [4]:
raw_data

,mouse_id,day,event,sensor,time,trial,signal
0,142_237,1,avoid,D1,-25.000000,0,0.955051
1,142_237,1,avoid,D1,-24.998034,0,0.956232
2,142_237,1,avoid,D1,-24.996067,0,0.957714
3,142_237,1,avoid,D1,-24.994102,0,0.959506
4,142_237,1,avoid,D1,-24.992136,0,0.961616
...,...,...,...,...,...,...,...
22884,313_255,7,shock,DA,19.992136,0,0.741634
22885,313_255,7,shock,DA,19.994102,0,0.713201
22886,313_255,7,shock,DA,19.996067,0,0.684301
22887,313_255,7,shock,DA,19.998034,0,0.654951


In [11]:
raw_data_us = raw_data.assign(time =pd.to_timedelta(raw_data['time'], unit='seconds'))
resample = raw_data_us.resample('100ms', on='time').mean()
resample

: 

: 

mouse_id     object
day           int64
event        object
sensor       object
time        float64
trial         int32
signal      float32
dtype: object